In [ ]:
# imports
from subprocess import check_output
import pandas as pd
import plotly.express as px
import plotly
import re
import nbformat
import time

In [ ]:
# run code for each p value repeat times from 0.1 to 0.9
scores = []
repeat = 20

for i in range(10):
    count = 0
    p = 0.1 * i
    for x in range(repeat):
        command = 'python main.py -d 101 -a a_star -p {}'.format(p)
        out = str(check_output(command, cwd='/Users/advithchegu/Desktop/Rutgers/Fall 2021/AI/Assignment 1', shell=True))
        if 'No Path Found' in out:
            continue
        else:
            count += 1
    count /= repeat
    scores.append(count)

print(scores)

In [ ]:
# conver the scores list to a dict
scores_dict = dict(zip([0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9], scores))
scores_df = pd.DataFrame.from_dict(scores_dict, orient='index').reset_index()
scores_df.columns = ["probability", "solvability"]
# plot the scores array

fig = px.line(scores_df, x="probability", y="solvability", title='Plot Density vs Solvability')
fig.show()

plotly.io.write_image(fig, file="./graphs/question_4.png", scale=5)

In [ ]:
# measure heuristic efficiency
times_df = pd.DataFrame(columns = ['Time', 'Heuristic'])

for x in range(100):
    command = 'python main.py -d 500 -a a_star -m all -p 0.4'
    out = str(check_output(command, cwd='/Users/advithchegu/Desktop/Rutgers/Fall 2021/AI/Assignment 1', shell=True))
    formatted = out.rstrip("]\\n'").lstrip("b'[").split(', ')
    list(map(float, formatted))

    times_df = times_df.append([{'Time': formatted[0], 'Heuristic': 'Chebyshev'}, {'Time': formatted[1], 'Heuristic': 'Manhattan'}, {'Time': formatted[2], 'Heuristic': 'Euclidian'}], ignore_index=True)

    # print(formatted)


print(times_df)

In [ ]:
# plot the box plots
fig = px.box(times_df, x="Heuristic", y="Time", title="Heuristics vs Time")
fig.show()
plotly.io.write_image(fig, file="./graphs/question_5.jpg", scale=5)

In [ ]:
# Trajectory Length: 201\n
traj_df = pd.DataFrame(columns = ['Density', 'Trajectory Length', 'Trial'])

for i in range(5):
    for x in range(34):
        p = 0.01*x
        command = 'python main.py -d 101 -a repeated_a_star -m all -p ' + str(p)
        out = str(check_output(command, cwd='/Users/advithchegu/Desktop/Rutgers/Fall 2021/AI/Assignment 1', shell=True))
        formatted = int(re.search(r"Trajectory Length: \d+", out).group().lstrip("Trajectory Length: "))
        while(formatted < 200):
            out = str(check_output(command, cwd='/Users/advithchegu/Desktop/Rutgers/Fall 2021/AI/Assignment 1', shell=True))
            formatted = int(re.search(r"Trajectory Length: \d+", out).group().lstrip("Trajectory Length: "))
        
        traj_df = traj_df.append([{'Density': p, 'Trajectory Length': formatted, 'Trial': i+1}])
print(traj_df)

In [98]:
avg_df = traj_df['Trajectory Length'].groupby(traj_df['Density'])
avg_df = avg_df.mean()
avg_df = avg_df.to_frame().reset_index()
print(avg_df.columns)
# avg_df['Density'].apply(pd.DataFrame)

Index(['Density', 'Trajectory Length'], dtype='object')


In [101]:

fig = px.line(avg_df, x="Density", y="Trajectory Length", title='Density vs Average Trajectory Length')
fig.show()

plotly.io.write_image(fig, file="./graphs/question_6_1.png", scale=5)